In [1]:
import os
import re
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.chemkin import loadChemkinFile
from rmgpy.molecule.draw import MoleculeDrawer
from rmgpy.chemkin import getSpeciesIdentifier
from rmgpy.rmg.output import OutputError
from extractInfoFromckcsv import getConcentrationDictFromCKCSV, getROPFromCKCSV
from pychemkin.chemkin import getTotalMoles

import numpy
from utilities.wikiMagics import JinjaMagics
from IPython.core.magic import register_cell_magic
ip = get_ipython()
ip.register_magics(JinjaMagics)

In [2]:
# chemkin model name
mech = 'pdd_scratch_add6'

In [3]:
path = os.path.abspath('./')
mechPath = path + '/data/' + mech
chemkinPath= mechPath + '/chem.inp'
dictionaryPath = mechPath + '/species_dictionary.txt'

model = CoreEdgeReactionModel()
model.core.species, model.core.reactions = loadChemkinFile(chemkinPath,dictionaryPath, \
                                                           readComments=False, identifyDuplicateReactions=False)

#generate species images
speciesPath = mechPath + '/species/'
if not os.path.isdir(speciesPath):
    os.makedirs(speciesPath)

species = model.core.species[:]
re_index_search = re.compile(r'\((\d+)\)$').search
for spec in species:
    match = re_index_search(spec.label)
    if match:
        spec.index = int(match.group(0)[1:-1])
        spec.label = spec.label[0:match.start()]
    # Draw molecules if necessary
    fstr = os.path.join(mechPath, 'species', '{0}.png'.format(spec))
    if not os.path.exists(fstr):
        try:
            MoleculeDrawer().draw(spec.molecule[0], 'png', fstr)
        except IndexError:
            raise OutputError("{0} species could not be drawn!".format(getSpeciesIdentifier(spec)))

In [4]:
ckcsvPath= mechPath + '/CKSoln.ckcsv'
firstColDict, spc_mf_dict = getConcentrationDictFromCKCSV(ckcsvPath)

_, spc_total_rop_dict, spc_rop_dict = getROPFromCKCSV(ckcsvPath)

totalMoles = getTotalMoles(ckcsvPath)[0]

In [5]:
# Give a reactant and conversion point for investigation
rxt = 'C18H30(3)'
# conv_investigated = 0.2
time_investigated = 0.01 #hr

# choose a spc for its ROP analysis
spc = 'C18H29(14)'# Toluene(3) or C8H8(9)

In [6]:
# get the top 20 important reactions for spc's rop at conv_investigated
initial_rxt_moles = spc_mf_dict[rxt][0]*totalMoles[0]

# idx = (numpy.abs(spc_mf_dict[rxt]*totalMoles-initial_rxt_moles*(1-conv_investigated))).argmin()
idx = (numpy.abs(firstColDict['Time_(sec)']-0.001*3600)).argmin()

spc_rxn_flux_sorted = sorted(spc_rop_dict[spc],key=lambda tup: -abs(tup[1][idx]))

top_num = 10
display_top_num = min(top_num, len(spc_rxn_flux_sorted))

top_rxnNum_list = []
top_flux_list = []
top_reactions = []

for i in range(display_top_num):
    header = spc_rxn_flux_sorted[i][0]
    rxnNum = int(header.split("#")[1].split('_')[0])
    flux = spc_rxn_flux_sorted[i][1][idx]
    top_rxnNum_list.append(rxnNum)
    top_flux_list.append(flux)
    top_reactions.append(model.core.reactions[rxnNum-1])

reactions = top_reactions

In [7]:
%%jinja html
rop_rxn.html

In [8]:
firstColDict['Time_(sec)'][idx]

3.1038920000000001